In [13]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler

df = pd.read_csv('datasets/example.csv', delimiter=';')
print(f"df shape: {df.shape}")

df shape: (4, 7)


In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

X = df.drop(columns=["diagnosis"])
y = df["diagnosis"]

scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2)


# Clustering with K-Means

In [ ]:
from sklearn.cluster import KMeans

kmeans = KMeans(n_clusters=3)
clusters = kmeans.fit_predict(X_scaled)

df["kmeans_cluster"] = clusters

In [ ]:
sns.scatterplot(data=df, x="pca1", y="pca2", hue="kmeans_cluster", palette="Set2")
plt.title("K-Means Clusters in PCA Space")

In [ ]:
sns.scatterplot(data=df, x="pca1", y="pca2", hue="diagnosis", palette="Set1")
plt.title("True Diagnoses in PCA Space")